In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import pandas as pd
import xarray as xr

In [4]:
from ecephys_analyses.data import paths
from ecephys.scoring import load_datetime_hypnogram, filter_states
from ecephys.utils import load_df_h5, replace_outliers
from ecephys.signal.sharp_wave_ripples import get_events_in_interval
import ecephys.plot as eplt
from neurodsp.plts.time_series import plot_time_series

In [5]:
from ripple_detection.core import gaussian_smooth

def my_gaussian_smooth(da, smoothing_sigma=10):
    estimated_fs = 1 / np.diff(da.time.values).mean()
    return gaussian_smooth(da, smoothing_sigma, estimated_fs)

In [6]:
def concatenate_power(bandpowers):
    for i in range(1, len(bandpowers)):
        bandpowers[i]['time'] = bandpowers[i].time + bandpowers[i-1].time.max()
        
    return xr.concat(bandpowers, dim="time")

In [76]:
subject = "Allan"
condition = "dark-period-circadian-match"

In [77]:
hypnogram_paths = paths.get_sglx_style_datapaths(subject=subject, experiment="sleep-homeostasis", condition=condition, ext="hypnogram.tsv")
hypnograms = [load_datetime_hypnogram(path) for path in hypnogram_paths]

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/neuropixel/Data/CNPIX8-Allan/3-1-2021/3-1-2021_D_g0/3-1-2021_D_g0_imec1/3-1-2021_D_g0_t0.imec1.hypnogram.tsv'

In [78]:
hypnogram = pd.concat(hypnograms).reset_index(drop=True)
condition_start_dt = hypnogram['start_time'].iloc[0]

In [10]:
spw_paths = paths.get_sglx_style_datapaths(subject=subject, experiment="sleep-homeostasis", condition=condition, ext="spws.h5")
spws = [load_df_h5(path) for path in spw_paths]

In [11]:
for _spws, _hypnogram in zip(spws, hypnograms):
    file_start_dt = _hypnogram['start_time'].min()
    if _spws.empty: 
        continue
    _spws['start_time'] = file_start_dt + pd.to_timedelta(_spws['start_time'], 's')
    _spws['end_time'] = file_start_dt + pd.to_timedelta(_spws['end_time'], 's')
    _spws['midpoint'] = file_start_dt + pd.to_timedelta(_spws['midpoint'], 's')
    #spws['duration'] = pd.to_timedelta(spws['duration'], 's')
    _spws['time_from_condition_start'] = _spws['start_time'] - condition_start_dt
    
combined_spws = pd.concat(spws).reset_index(drop=True)
combined_spws.index += 1
combined_spws.index = combined_spws.index.rename('spw_number')
spws = combined_spws

In [12]:
bandpower_paths = paths.get_sglx_style_datapaths(subject=subject, experiment="sleep-homeostasis", condition=condition, ext="pow.nc")
bandpowers = [xr.open_dataset(path) for path in bandpower_paths]
bandpower = concatenate_power(bandpowers)

In [68]:
hypnogram["end_time"].iloc[-1] = spws["end_time"].iloc[-1]
mask = (hypnogram['start_time'].min() + pd.to_timedelta(bandpower.time.values, 's')) < spws["end_time"].iloc[-1]
bandpower = bandpower.isel(time=mask)

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [71]:
mpta_swa = my_gaussian_smooth(bandpower.mpta_delta, smoothing_sigma=10)
mpta_swa_dt = pd.date_range(min(hypnogram.start_time), max(hypnogram.end_time), periods=len(mpta_swa))

In [72]:
sr_swa = my_gaussian_smooth(bandpower.sr_delta, smoothing_sigma=10)
sr_swa_dt = pd.date_range(min(hypnogram.start_time), max(hypnogram.end_time), periods=len(sr_swa))

In [73]:
# density = spws.set_index('start_time').resample('10s').count().mean(axis=1).to_frame(name='frequency').reset_index()

In [74]:
fig, (mpta_swa_ax, sr_swa_ax, density_ax, duration_ax, amplitude_ax, integral_ax) = plt.subplots(6, 1, figsize=(22,12))

xmargin=0

sns.lineplot(x=mpta_swa_dt, y=mpta_swa, color='black', ax=mpta_swa_ax)
mpta_swa_ax.set(xlabel=None, ylabel='CTX SWA (uV^2/Hz)', ylim=(0, 20000), xmargin=xmargin, xticks=[])

sns.lineplot(x=sr_swa_dt, y=sr_swa, color='black', ax=sr_swa_ax)
sr_swa_ax.set(xlabel=None, ylabel='HPC SWA (uV^2/Hz)', ylim=(0, 80000), xmargin=xmargin, xticks=[])

binwidth = 10
nbins = np.int((spws.start_time.max() - spws.start_time.min()).total_seconds() / binwidth)
sns.histplot(data=spws, x='start_time', bins=nbins, color='black', ax=density_ax)
density_ax.set(xlabel=None, ylabel='Frequency (Hz)', xmargin=xmargin, xticks=[])

sns.scatterplot(data=spws, x="start_time", y="duration", s=2, ax=duration_ax)
duration_ax.set(xlabel=None, ylabel="Duration (s)", xmargin=xmargin, xticks=[])

sns.scatterplot(data=spws, x='start_time', y='sink_amplitude', s=2, ax=amplitude_ax)
amplitude_ax.set(xlabel=None, ylabel="Sink amplitude (mA/mm)", xmargin=xmargin, xticks=[])

sns.scatterplot(data=spws, x='start_time', y='sink_integral', s=2, ax=integral_ax)
integral_ax.set(xlabel="Event time (month-date hour)", ylabel="Sink integral (mA * ms)", xmargin=xmargin)

for ax in (mpta_swa_ax, sr_swa_ax, density_ax, duration_ax, amplitude_ax, integral_ax):
    eplt.plot_hypnogram_overlay(hypnogram, ax=ax)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [95]:
def add_spw_properties_to_hypnogram(spws, hypnogram):
    for i, bout in hypnogram.iterrows():
        bout_events = get_events_in_interval(spws, bout.start_time, bout.end_time)
        hypnogram.at[i, 'spw_density'] = len(bout_events) / bout.duration.total_seconds()
        hypnogram.at[i, 'mean_duration'] = bout_events.duration.mean()
        hypnogram.at[i, 'mean_sink_amplitude'] = bout_events.sink_amplitude.mean()
        hypnogram.at[i, 'mean_sink_integral'] = bout_events.sink_integral.mean()
    
    hypnogram.spw_density.fillna(value=0, inplace=True)
    hypnogram.mean_duration.fillna(value=0, inplace=True)
    hypnogram.mean_sink_amplitude.fillna(value=0, inplace=True)
    hypnogram.mean_sink_integral.fillna(value=0, inplace=True)

In [96]:
add_spw_properties_to_hypnogram(spws, hypnogram)

In [97]:
qwk = filter_states(hypnogram, ['qWk']).reset_index(drop=True)

In [45]:
def plot_spw_summary_by_bout_rank(hypnogram, figsize=(20, 8)):
    fig, (density_ax, duration_ax, amplitude_ax, integral_ax) = plt.subplots(4, 1, figsize=figsize)
    
    hypnogram.plot.scatter(x='index', y='spw_density', ax=density_ax, title='SPW density (Hz)', s=1)
    hypnogram.plot.scatter(x='index', y='mean_duration', ax=duration_ax, title='Event duration (s)', s=1)
    hypnogram.plot.scatter(x='index', y='mean_sink_amplitude', ax=amplitude_ax, title='Sink amplitude (mA/mm)', s=1)
    hypnogram.plot.scatter(x='index', y='mean_sink_integral', ax=integral_ax, title='Sink integral (mA * ms)', s=1)
    for ax in (density_ax, duration_ax, amplitude_ax, integral_ax):
        ax.set_xlabel('Bout rank')
    plt.tight_layout()

In [46]:
plot_spw_summary_by_bout_rank(qwk.reset_index())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
awk = filter_states(hypnogram, ['aWk']).reset_index(drop=True)

In [48]:
plot_spw_summary_by_bout_rank(awk.reset_index())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …